<a href="https://colab.research.google.com/github/adharv49/project/blob/main/gitside_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/adharv49/project.git

In [ ]:
!pip install diffusers accelerate

In [ ]:
from diffusers import AutoPipelineForInpainting, AutoencoderKL
from diffusers.utils import load_image
import torch

In [ ]:
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipeline = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", vae=vae, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to("cuda")
pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin", low_cpu_mem_usage=True)

In [ ]:
!pip install albumentations
!pip install iglovikov-helper-functions
!pip install cloths_segmentation
!pip install onnxruntime
!pip install gradio
!pip install insightface

In [ ]:
%cd project

In [ ]:
from SegBody import segment_body
from diffusers.utils import load_image
import numpy as np
import cv2
import torch
import albumentations as albu
from PIL import Image
from iglovikov_helper_functions.utils.image_utils import load_rgb, pad, unpad
from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image
#from cloths_segmentation.pre_trained_models import create_model
from diffusers import AutoencoderKL,AutoPipelineForInpainting

def generate_masked_image(cloth_path, model_path):
    cloth_image = load_image(cloth_path).convert("RGB")
    cloth_image.resize((512, 512))
    model_image = load_image(model_path).convert("RGB")
    model_image.resize((512, 512))
    seg_image, mask_image = segment_body(model_path, face=False)
    mask_image.resize((512, 512))
    pipeline.set_ip_adapter_scale(1.0)
    images = pipeline(
    prompt="photorealistic, perfect body, beautiful skin, realistic skin, natural skin",
    negative_prompt="ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet,wrong alphabets, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings",
    image=model_image,
    mask_image=mask_image,
    ip_adapter_image=cloth_image,
    strength=0.99,
    guidance_scale=7.5,
    num_inference_steps=100,
    ).images
    return images[0]

In [ ]:
import gradio as gr
from PIL import Image
import os
def run(cloth, model):
    # Load the selected cloth and the uploaded model image
    #cloth = Image.open(cloth_path)
    model.save("temp_model.jpg")
    cloth.save("temp_cloth.jpg")

    # Call the masking function from mask.py
    # Replace generate_masked_image with your actual function
    masked_image = generate_masked_image("temp_cloth.jpg", "temp_model.jpg")

    # Return the masked image for display
    return masked_image

# Predefined cloth images
CLOTH_OPTIONS = {
    "Cloth 1": "/content/project/samplecloth/01_upper.jpg",
    "Cloth 2": "/content/project/samplecloth/06_upper.png",
    "Cloth 3": "/content/project/samplecloth/09163_00.jpg",
    "Cloth 4": "/content/project/samplecloth/09176_00.jpg",
    "Cloth 5": "/content/project/samplecloth/09236_00.jpg",
    "Cloth 6": "/content/project/samplecloth/09290_00.jpg",
    "Cloth 7": "/content/project/samplecloth/boys-Puffer-Coat-With-Detachable-Hood-1.jpg",
    "Cloth 8": "/content/project/samplecloth/brown-jacket.jpg",
    "Cloth 9": "/content/project/samplecloth/clothing1.jpg",
    "Cloth 10": "/content/project/samplecloth/puma-hoodie.jpg",
    "Cloth 24": "/content/project/samplecloth/black-hoodie.jpg"

}

# Gradio app
with gr.Blocks() as app:
    with gr.Row():
        gr.Markdown("<center><h1> DIGIDRAPE </h1><center>")

    with gr.Row():
        with gr.Column():
            # Dropdown to select predefined cloths

            cloth_input=gr.Image(sources='clipboard', type="pil", label="Model", value=CLOTH_OPTIONS["Cloth 1"])
            example = gr.Examples(inputs=cloth_input,
                                  examples_per_page=4,
                                  examples=[CLOTH_OPTIONS["Cloth 1"], CLOTH_OPTIONS["Cloth 2"],CLOTH_OPTIONS["Cloth 3"],
                                            CLOTH_OPTIONS["Cloth 4"],CLOTH_OPTIONS["Cloth 5"],CLOTH_OPTIONS["Cloth 6"],
                                            CLOTH_OPTIONS["Cloth 7"],CLOTH_OPTIONS["Cloth 8"],CLOTH_OPTIONS["Cloth 9"],
                                            CLOTH_OPTIONS["Cloth 10"],CLOTH_OPTIONS["Cloth 24"]])
        with gr.Column():
            model_input = gr.Image(sources=['upload'], type="pil", label="Upload the Human Image")

    with gr.Row():
        final_output = gr.Image(type="pil", label="Final Prediction")

    with gr.Row():
        submit_button = gr.Button("Submit")



    submit_button.click(fn=run, inputs=[cloth_input, model_input], outputs=[final_output])

app.launch(debug=True, share=True)


In [ ]:
import gradio as gr
from PIL import Image
import os
CLOTH_OPTIONS = {
    "Cloth 101": "/content/project/samplecloth/01_upper.jpg",
    "Cloth 102": "/content/project/samplecloth/tantra.jpg",
    "Cloth 103": "/content/project/samplecloth/09163_00.jpg",
    "Cloth 104": "/content/project/samplecloth/09176_00.jpg",
    "Cloth 105": "/content/project/samplecloth/09236_00.jpg",
    "Cloth 106": "/content/project/samplecloth/09290_00.jpg",
     "Cloth 107": "/content/project/samplecloth/clothing1.jpg",
    "Cloth 108": "/content/project/samplecloth/brown-jacket.jpg",
    "Cloth 109": "/content/project/samplecloth/boys-Puffer-Coat-With-Detachable-Hood-1.jpg",
    "Cloth 110": "/content/project/samplecloth/boys-Puffer-Coat-With-Detachable-Hood-2.jpg",
   "Cloth 111":"/content/g-polos-tshirt-2.png",
    "Cloth 112":"/content/project/samplecloth/tantra.jpg",
    "Cloth 113": "/content/pink jacket.jpg",
    "Cloth 115":"/content/Man-Geox-Winter-jacket-1.jpg",
    "Cloth 116": "/content/project/samplecloth/black-hoodie.jpg",

}
def convert_png_to_jpg(input_path, output_path):
    try:
        # Open the PNG image
        if isinstance(input_path, str):
            # Open the PNG image if input_path is a file path
            image = Image.open(input_path)
        elif isinstance(input_path, Image.Image):
            # Use the existing Image object
            image = input_path
        #image = Image.open(input_path)

        # Convert RGBA (if present) to RGB
        if image.mode == 'RGBA':
            image = image.convert("RGB")

        # Save as JPG
        image.save(output_path, format="JPEG")
    finally:
        print("Conversion completed.")
def run(request: gr.Request,cloth_key, model):
    query_params = request.query_params
    cloth_key = query_params.get("cloth", "Cloth 1")
    image_path =CLOTH_OPTIONS.get(cloth_key, None)
    cloth= Image.open(image_path)
    '''if not cloth_image:
        raise ValueError(f"Invalid cloth key: {cloth_key}")'''

    # Load the cloth image
    #cloth = Image.open(cloth_path)
    # Load the selected cloth and the uploaded model image
    #cloth = Image.open(cloth_path)
    model.save("temp_model.jpg")
    #cloth.save("temp_cloth.jpg")
    output_file = "temp_cloth2.jpg"  # Desired output file path
    convert_png_to_jpg(input_path=cloth, output_path=output_file)
    # Call the masking function from mask.py
    # Replace generate_masked_image with your actual function
    masked_image = generate_masked_image("temp_cloth2.jpg", "temp_model.jpg")

    # Return the masked image for display
    return cloth,masked_image
with gr.Blocks() as demo:
    gr.Markdown("# DigiDrape Virtual Try-On System")

    # Input for cloth key
    cloth_key_input = gr.Textbox(label="Cloth Key", value=None, visible=False)
    image_path = CLOTH_OPTIONS.get(cloth_key_input, None)
    cloth_image = gr.Image(value=image_path, visible=False)


    with gr.Row():
        # Image display for cloth
        cloth_image_output = gr.Image(label="Selected Cloth")

        # Upload human image
        human_image_input = gr.Image(label="Upload Human Image", type="pil")
        # Combined result display
        combined_image_output = gr.Image(label="Combined Result")

    # Button to trigger processing
    submit_button = gr.Button("Combine and Display")

    # Set up the function for the interface
    submit_button.click(
        run,
        inputs=[cloth_key_input, human_image_input],
        outputs=[cloth_image_output, combined_image_output],
    )

demo.launch(share=True,debug=True)
